## Hyperparameter search

### Loading and Preparing Data

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display
from scipy.stats import pearsonr, spearmanr, kendalltau

method_to_func = {"pearson": pearsonr, "kendall": kendalltau, "spearman": spearmanr}

def build_cols(metrics):
    """ Columns for df """
    columns = []
    for x in metrics:
        columns += [f"{x}_corr", f"{x}_pval"]
    return columns

def get_correlations(df, method, params, metrics):
    """ Retrieve correlations between params and metrics """
    data, mappings = [], {}
    for p in params:
        curr_data = []
        for m in metrics:
            if all (isinstance(x, float) for x in df[p].values):
                x = list(df[p].values)
            else:
                curr_mapping = {val: index for index, val in enumerate(df[p].unique())}
                mappings[p] = curr_mapping
                x = [curr_mapping[elt] for elt in df[p].values]
            curr_data += list(method_to_func[method](x, list(df[m].values)))
        data.append(curr_data)
    return pd.DataFrame(data, columns=build_cols(metrics), index=params), mappings

In [2]:
df = pd.read_csv("hp_search_results.csv", index_col=0).fillna("none")
df.head(3)

,summary_method,summary_percentage,ranking,ranking_how,ranking_perc_threshold,entity,relation,meteor_pr,meteor_re,meteor_f1,rouge-2_pr,rouge-2_re,rouge-2_f1
15,none,none,page\_rank,all,15.0,dbpedia\_spotlight,corenlp,37.0,21.2,26.2,1.8,23.6,3.0
116,none,none,page\_rank,all,15.0,dbpedia\_spotlight,rebel\_hf,35.3,20.4,25.4,2.1,23.7,3.8
129,none,none,page\_rank,all,15.0,dbpedia\_spotlight,rebel\_ft,33.0,21.4,25.4,3.6,21.6,6.0


In [3]:
params = [x for x in df.columns if not any(x.startswith(y) for y in ["rouge", "meteor", "avg"])]
# params_summary_ranking = [x for x in df.columns if any(x.startswith(y) for y in ["summary", "ranking"])]
# df["params_sr"] = df.apply(lambda row: "_".join(str(row[x]) for x in params_summary_ranking), axis=1)
# params += ["params_sr"]
metrics = [x for x in df.columns if any(x.startswith(y) for y in ["rouge", "meteor"])]
for p in params:
    print(f"{p}\t{df[p].unique()}")
for m in metrics:
    print(f"{m}\t{np.min(df[m].values)}/{round(np.mean(df[m].values), 2)}/{np.max(df[m].values)}")
for val in ["pr", "re", "f1"]:
    df[f"avg_{val}"] = (df[f"meteor_{val}"] + df[f"rouge-2_{val}"])/2
df.head(3)

summary_method	['none' 'chat-gpt' 'lex-rank']
summary_percentage	['none' 15.0 30.0]
ranking	['page\\_rank' 'word2vec' 'none']
ranking_how	['all' 'none']
ranking_perc_threshold	[15.0 30.0 'none']
entity	['dbpedia\\_spotlight' 'nps']
relation	['corenlp' 'rebel\\_hf' 'rebel\\_ft']
meteor_pr	12.0/27.61/44.6
meteor_re	13.0/21.87/34.5
meteor_f1	12.5/23.02/30.7
rouge-2_pr	0.4/4.5/12.7
rouge-2_re	2.3/15.67/36.3
rouge-2_f1	0.9/5.26/10.5


,summary_method,summary_percentage,ranking,ranking_how,ranking_perc_threshold,entity,relation,meteor_pr,meteor_re,meteor_f1,rouge-2_pr,rouge-2_re,rouge-2_f1,avg_pr,avg_re,avg_f1
15,none,none,page\_rank,all,15.0,dbpedia\_spotlight,corenlp,37.0,21.2,26.2,1.8,23.6,3.0,19.4,22.40,14.6
116,none,none,page\_rank,all,15.0,dbpedia\_spotlight,rebel\_hf,35.3,20.4,25.4,2.1,23.7,3.8,18.7,22.05,14.6
129,none,none,page\_rank,all,15.0,dbpedia\_spotlight,rebel\_ft,33.0,21.4,25.4,3.6,21.6,6.0,18.3,21.50,15.7


### Correlations for entity + relations

In [4]:
corrs, mappings = get_correlations(df, 'pearson', ['entity'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
corrs

entity:	{'dbpedia\\_spotlight': 0, 'nps': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
entity,-0.482235,1.226712e-09,-0.542095,3.228761e-12,-0.635097,2.095694e-17


In [5]:
corrs, mappings = get_correlations(df[df.relation.isin(['corenlp', 'rebel\_ft'])], 'pearson', ['relation'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
display(corrs)

corrs, mappings = get_correlations(df[df.relation.isin(['corenlp', 'rebel\_hf'])], 'pearson', ['relation'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
display(corrs)

corrs, mappings = get_correlations(df[df.relation.isin(['rebel\_hf', 'rebel\_ft'])], 'pearson', ['relation'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
display(corrs)

relation:	{'corenlp': 0, 'rebel\\_ft': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
relation,0.108574,0.297567,0.030114,0.773253,0.368055,0.000263


relation:	{'corenlp': 0, 'rebel\\_hf': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
relation,0.193341,0.061889,0.190373,0.066079,0.383837,0.000134


relation:	{'rebel\\_hf': 0, 'rebel\\_ft': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
relation,-0.101366,0.325753,-0.200435,0.050224,0.012486,0.903898


Params
* `entity = "dbpedia\\_spotlight"`
* `relation = "rebel\\_hf"`

In [6]:
# Only keeping dbpedia spotlight for entity
df_filter = df[(df.entity == 'dbpedia\\_spotlight') & (df.relation == "rebel\_hf")]
print(df_filter.shape)
df_filter.head(3)

(24, 16)


,summary_method,summary_percentage,ranking,ranking_how,ranking_perc_threshold,entity,relation,meteor_pr,meteor_re,meteor_f1,rouge-2_pr,rouge-2_re,rouge-2_f1,avg_pr,avg_re,avg_f1
116,none,none,page\_rank,all,15.0,dbpedia\_spotlight,rebel\_hf,35.3,20.4,25.4,2.1,23.7,3.8,18.70,22.05,14.60
60,none,none,page\_rank,all,30.0,dbpedia\_spotlight,rebel\_hf,39.0,18.9,25.1,1.3,29.2,2.4,20.15,24.05,13.75
121,none,none,word2vec,all,15.0,dbpedia\_spotlight,rebel\_hf,34.2,17.6,22.8,1.8,22.7,3.3,18.00,20.15,13.05


### Summary + Importance Ranking

In [24]:
curr_df = df_filter[(df_filter.summary_method != "none") & (df_filter.ranking != "none")]
corrs, mappings = get_correlations(curr_df, 'pearson', ['summary_method', 'summary_percentage', 'ranking', 'ranking_perc_threshold'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
corrs

summary_method:	{'chat-gpt': 0, 'lex-rank': 1}
ranking:	{'page\\_rank': 0, 'word2vec': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
summary_method,-0.556913,0.025035,-0.629487,0.008977,-0.917694,5.507472e-07
summary_percentage,0.207607,0.440394,0.403822,0.120856,-0.051860,8.487297e-01
ranking,-0.082384,0.761647,-0.154402,0.568032,-0.142051,5.997319e-01
ranking_perc_threshold,0.570095,0.021127,0.451330,0.079297,-0.060879,8.227814e-01


In [25]:
print('avg_f1')
display(curr_df[curr_df['avg_f1']==np.max(curr_df['avg_f1'].values)][['summary_method', 'summary_percentage', 'ranking', 'ranking_perc_threshold']])

avg_f1


,summary_method,summary_percentage,ranking,ranking_perc_threshold
136,chat-gpt,15.0,word2vec,15.0


* `summary_method = "chat-gpt"` (significant)
* `summary_percentage = 15` (best, not significant)
* `ranking = "word2vec"`(best, not significant)
* `ranking_perc_threshold = 15` (best, not significant)

In [26]:
df_filter[(df_filter.summary_method == "chat-gpt") & (df_filter.summary_percentage == 15) & \
    (df_filter.ranking == "word2vec") & (df_filter.ranking_perc_threshold == 15)][metrics]

,meteor_pr,meteor_re,meteor_f1,rouge-2_pr,rouge-2_re,rouge-2_f1
136,26.3,28.8,26.8,9.4,13.5,10.3


### Summary Only

In [27]:
curr_df = df_filter[(df_filter.summary_method != "none") & (df_filter.ranking == "none")]
corrs, mappings = get_correlations(curr_df, 'pearson', ['summary_method', 'summary_percentage'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
corrs

summary_method:	{'chat-gpt': 0, 'lex-rank': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
summary_method,0.546263,0.453737,-0.500433,0.499567,-0.962876,0.037124
summary_percentage,0.667655,0.332345,0.706494,0.293506,-0.262603,0.737397


In [28]:
print('avg_f1')
display(curr_df[curr_df['avg_f1']==np.max(curr_df['avg_f1'].values)][['summary_method', 'summary_percentage']])

avg_f1


,summary_method,summary_percentage
16,chat-gpt,15.0


* `summary_method = "chat-gpt"` (significant for f1 but not others)
* `summary_percentage = 15` (best, not significant)

In [29]:
curr_df[(curr_df.summary_method == "chat-gpt") & (curr_df.summary_percentage == 15)][metrics]

,meteor_pr,meteor_re,meteor_f1,rouge-2_pr,rouge-2_re,rouge-2_f1
16,34.6,23.1,27.0,3.2,23.6,5.4


### Ranking Only

In [32]:
curr_df = df_filter[(df_filter.summary_method == "none") & (df_filter.ranking != "none")]
corrs, mappings = get_correlations(curr_df, 'pearson', ['ranking', 'ranking_perc_threshold'], ["avg_pr", "avg_re", "avg_f1"])
for k, v in mappings.items():
    print(f"{k}:\t{v}")
corrs

ranking:	{'page\\_rank': 0, 'word2vec': 1}


,avg_pr_corr,avg_pr_pval,avg_re_corr,avg_re_pval,avg_f1_corr,avg_f1_pval
ranking,-0.360342,0.639658,-0.599482,0.400518,-0.891338,0.108662
ranking_perc_threshold,0.930885,0.069115,0.796327,0.203673,-0.371391,0.628609


In [33]:
print('avg_f1')
display(curr_df[curr_df['avg_f1']==np.max(curr_df['avg_f1'].values)][['ranking', 'ranking_perc_threshold']])

avg_f1


,ranking,ranking_perc_threshold
116,page\_rank,15.0


* `ranking = "page\\_rank"`(not significant)
* `ranking_perc_threshold = 15` (significant for precision)

In [34]:
curr_df[(curr_df.ranking == 'page\\_rank') & (curr_df.ranking_perc_threshold == 15)][metrics]

,meteor_pr,meteor_re,meteor_f1,rouge-2_pr,rouge-2_re,rouge-2_f1
116,35.3,20.4,25.4,2.1,23.7,3.8
